In [30]:
# Import Package
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Data Preparation

In [31]:
# Pre-treatment Data: Signing up
des=pd.read_stata("oregonhie_descriptive_vars.dta")
des.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74922 entries, 0 to 74921
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   person_id          74922 non-null  float32       
 1   household_id       74922 non-null  float32       
 2   treatment          74922 non-null  category      
 3   draw_treat         29834 non-null  category      
 4   draw_lottery       74922 non-null  category      
 5   applied_app        29799 non-null  category      
 6   approved_app       29799 non-null  category      
 7   dt_notify_lottery  74922 non-null  category      
 8   dt_retro_coverage  74922 non-null  datetime64[ns]
 9   dt_app_decision    18101 non-null  datetime64[ns]
 10  postn_death        74922 non-null  category      
 11  numhh_list         74922 non-null  category      
 12  birthyear_list     74922 non-null  int16         
 13  have_phone_list    74922 non-null  category      
 14  englis

In [32]:
# Keeping: demographical information: numhh_list/ birthyear_list(Older people tend to devote more into insurance while younger may not)/ female_list / english_list/ have_phone_list(not pretty sure though)/ first_day_list /pobox_list

# Dropping: household_id / information about lottery drawing / information about application / date*3 / poastn_death / week
des_2=des[["person_id","treatment","numhh_list","birthyear_list","female_list","english_list","have_phone_list","first_day_list","pobox_list","self_list"]]

des_2.head()

,person_id,treatment,numhh_list,birthyear_list,female_list,english_list,have_phone_list,first_day_list,pobox_list,self_list
0,1.0,Selected,signed self up,1978,0: Male,Requested English materials,Gave Phone Number,Did NOT sign up for lottery list on first day,1: POBOX,Signed self up
1,2.0,Selected,signed self up,1984,1: Female,Requested English materials,Gave Phone Number,Did NOT sign up for lottery list on first day,0: Not POBOX,Signed self up
2,3.0,Not selected,signed self up,1971,1: Female,Requested English materials,Gave Phone Number,Did NOT sign up for lottery list on first day,0: Not POBOX,Signed self up
3,4.0,Not selected,signed self up,1955,1: Female,Requested English materials,Gave Phone Number,Did NOT sign up for lottery list on first day,0: Not POBOX,Signed self up
4,5.0,Selected,signed self up,1969,1: Female,Requested materials in language other than eng...,Gave Phone Number,Did NOT sign up for lottery list on first day,0: Not POBOX,Signed self up


In [33]:
pd.options.mode.chained_assignment = None

In [34]:
# Dummy For the Lottery Selection Variable
des_2["treatment"]=des_2["treatment"].replace(to_replace='Selected',value=1)
des_2["treatment"]=des_2['treatment'].replace(to_replace='Not selected',value=0)
des_2['treatment']=pd.to_numeric(des_2['treatment'])
des_2.head()

,person_id,treatment,numhh_list,birthyear_list,female_list,english_list,have_phone_list,first_day_list,pobox_list,self_list
0,1.0,1,signed self up,1978,0: Male,Requested English materials,Gave Phone Number,Did NOT sign up for lottery list on first day,1: POBOX,Signed self up
1,2.0,1,signed self up,1984,1: Female,Requested English materials,Gave Phone Number,Did NOT sign up for lottery list on first day,0: Not POBOX,Signed self up
2,3.0,0,signed self up,1971,1: Female,Requested English materials,Gave Phone Number,Did NOT sign up for lottery list on first day,0: Not POBOX,Signed self up
3,4.0,0,signed self up,1955,1: Female,Requested English materials,Gave Phone Number,Did NOT sign up for lottery list on first day,0: Not POBOX,Signed self up
4,5.0,1,signed self up,1969,1: Female,Requested materials in language other than eng...,Gave Phone Number,Did NOT sign up for lottery list on first day,0: Not POBOX,Signed self up


In [35]:
# Same thing for household 
des_2['numhh_list'].value_counts()

signed self up                          57528
signed self up + 1 additional person    17236
signed self up + 2 additional people      158
Name: numhh_list, dtype: int64

In [36]:
des_2['numhh_list']=des_2['numhh_list'].replace(to_replace="signed self up",value=1)
des_2['numhh_list']=des_2['numhh_list'].replace(to_replace="signed self up + 1 additional person",value=2)
des_2['numhh_list']=des_2['numhh_list'].replace(to_replace = 'signed self up + 2 additional people',value=3)

des_2['english_list']=des_2['english_list'].replace(to_replace="Requested English materials",value=1)
des_2['english_list']=des_2['english_list'].replace(to_replace="Requested materials in language other than english",value=0)

des_2['female_list']=des_2['female_list'].replace(to_replace="0: Male",value=0)
des_2['female_list']=des_2['female_list'].replace(to_replace="1: Female",value=1)

des_2['have_phone_list']=des_2['have_phone_list'].replace(to_replace="Gave Phone Number",value=1)
des_2['have_phone_list']=des_2['have_phone_list'].replace(to_replace="Did NOT give phone number",value=0)

des_2['self_list']=des_2['self_list'].replace(to_replace='Signed self up',value=1)
des_2['self_list']=des_2['self_list'].replace(to_replace="Did NOT sign self up",value=0)

des_2['first_day_list']=des_2['first_day_list'].replace(to_replace="Signed up for lottery list on first day",value=1)
des_2['first_day_list']=des_2['first_day_list'].replace(to_replace="Did NOT sign up for lottery list on first day",value=0)

des_2['pobox_list']=des_2['pobox_list'].replace(to_replace="1: POBOX",value=1)
des_2['pobox_list']=des_2['pobox_list'].replace(to_replace="0: Not POBOX",value=0)

des_2['numhh_list']=pd.to_numeric(des_2['numhh_list'])
des_2['english_list'] =pd.to_numeric(des_2['english_list'])
des_2['female_list'] =pd.to_numeric(des_2['female_list'])
des_2['have_phone_list']=pd.to_numeric(des_2['have_phone_list'])
des_2['self_list'] =pd.to_numeric(des_2['self_list'])
des_2['first_day_list'] =pd.to_numeric(des_2['first_day_list'])
des_2['pobox_list'] =pd.to_numeric(des_2['pobox_list'])

des_2.head()

,person_id,treatment,numhh_list,birthyear_list,female_list,english_list,have_phone_list,first_day_list,pobox_list,self_list
0,1.0,1,1,1978,0.0,1,1,0,1,1
1,2.0,1,1,1984,1.0,1,1,0,0,1
2,3.0,0,1,1971,1.0,1,1,0,0,1
3,4.0,0,1,1955,1.0,1,1,0,0,1
4,5.0,1,1,1969,1.0,0,1,0,0,1


In [39]:
sp=pd.read_stata("oregonhie_stateprograms_vars.dta")
sp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74922 entries, 0 to 74921
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   person_id                      74922 non-null  float32 
 1   ohp_all_ever_matchn_30sep2009  74922 non-null  category
 2   ohp_all_ever_firstn_survey0m   58405 non-null  category
 3   ohp_all_ever_firstn_survey6m   13024 non-null  category
 4   ohp_all_ever_inperson          20745 non-null  category
 5   ohp_all_ever_firstn_30sep2009  74922 non-null  category
 6   ohp_all_end_30sep2009          74922 non-null  category
 7   ohp_all_end_survey0m           58405 non-null  category
 8   ohp_all_end_survey6m           13024 non-null  category
 9   ohp_all_end_inperson           20745 non-null  category
 10  ohp_all_at_12m                 58405 non-null  category
 11  ohp_all_mo_matchn_30sep2009    74922 non-null  category
 12  ohp_all_mo_firstn_survey0m     5

In [41]:
sp_2=sp[["person_id","ohp_all_ever_firstn_30sep2009"]]
sp_2=pd.get_dummies(sp_2,columns=["ohp_all_ever_firstn_30sep2009"],drop_first=True)
sp_2=sp_2.rename(columns={'ohp_all_ever_firstn_30sep2009_Enrolled': 'Enrollment'})
sp_2.head()

,person_id,Enrollment
0,1.0,0
1,2.0,1
2,3.0,0
3,4.0,1
4,5.0,0


In [43]:
edvar=pd.read_stata("oregonhie_ed_vars.dta")
edvar.head()

,person_id,sample_ed,any_visit_pre_ed,any_visit_ed,num_visit_pre_cens_ed,num_visit_cens_ed,any_hosp_pre_ed,any_hosp_ed,num_hosp_pre_cens_ed,num_hosp_cens_ed,...,ed_charg_tot_pre_ed,ed_charg_tot_ed,any_hiun_pre_ed,any_hiun_ed,num_hiun_pre_cens_ed,num_hiun_cens_ed,any_loun_pre_ed,any_loun_ed,num_loun_pre_cens_ed,num_loun_cens_ed
0,60562.0,1.0,No,No,0.0,0.0,No,No,0.0,0.0,...,0.0,0.000000,No,No,0.0,0.0,No,No,0.0,0.0
1,51142.0,1.0,No,Yes,0.0,3.0,No,No,0.0,0.0,...,0.0,2220.699951,No,Yes,0.0,3.0,No,No,0.0,0.0
2,60314.0,1.0,No,Yes,0.0,2.0,No,Yes,0.0,1.0,...,0.0,4529.549805,No,Yes,0.0,2.0,No,No,0.0,0.0
3,50902.0,1.0,No,No,0.0,0.0,No,No,0.0,0.0,...,0.0,0.000000,No,No,0.0,0.0,No,No,0.0,0.0
4,70733.0,1.0,No,No,0.0,0.0,No,No,0.0,0.0,...,0.0,0.000000,No,No,0.0,0.0,No,No,0.0,0.0


In [50]:
edvar_2=edvar[["person_id","any_visit_ed","num_visit_cens_ed"]]
edvar_2=edvar_2.rename(columns={'num_visit_cens_ed': 'num_visit','any_visit_ed':"any_visit"})
edvar_2['any_visit']=edvar_2['any_visit'].replace(to_replace="No",value=0)
edvar_2['any_visit']=edvar_2['any_visit'].replace(to_replace="Yes",value=1)
edvar_2['any_visit'] = pd.to_numeric(edvar_2['any_visit'])
edvar_2

,person_id,any_visit,num_visit
0,60562.0,0,0.0
1,51142.0,1,3.0
2,60314.0,1,2.0
3,50902.0,0,0.0
4,70733.0,0,0.0
...,...,...,...
24641,74030.0,1,18.0
24642,650.0,1,NaN
24643,60044.0,1,NaN
24644,2477.0,1,22.0


In [51]:
Joint=pd.merge(des_2,sp_2,how="inner",on="person_id")
Joint=pd.merge(Joint,edvar_2,how="inner",on="person_id")
Joint

,person_id,treatment,numhh_list,birthyear_list,female_list,english_list,have_phone_list,first_day_list,pobox_list,self_list,Enrollment,any_visit,num_visit
0,5.0,1,1,1969,1.0,0,1,0,0,1,0,0,0.0
1,8.0,0,2,1968,0.0,1,1,0,0,1,0,1,2.0
2,9.0,0,1,1977,1.0,1,1,0,0,1,0,0,0.0
3,16.0,0,2,1971,1.0,1,1,0,0,1,1,1,5.0
4,18.0,0,1,1957,0.0,1,1,0,0,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24641,74907.0,1,1,1953,0.0,1,1,0,0,1,0,0,0.0
24642,74911.0,1,2,1963,0.0,1,1,1,1,0,0,0,0.0
24643,74915.0,1,1,1951,1.0,0,1,1,1,1,1,0,0.0
24644,74918.0,0,1,1955,0.0,1,1,0,0,1,0,1,5.0


# Balance Check

Conduct Balance Check on demographical variables

In [55]:
year = sm.OLS(endog = Joint['birthyear_list'], exog = sm.add_constant(Joint[['treatment','numhh_list']]))
bal_ch_year = year.fit()
print(bal_ch_year.summary())
print("p-value is 0.543, we cannot reject the null hypothesis--> Balance")

                            OLS Regression Results                            
Dep. Variable:         birthyear_list   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.734
Date:                Thu, 20 May 2021   Prob (F-statistic):             0.0650
Time:                        20:23:19   Log-Likelihood:                -96304.
No. Observations:               24646   AIC:                         1.926e+05
Df Residuals:                   24643   BIC:                         1.926e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1967.8769      0.239   8218.458      0.0

In [56]:
gender = sm.OLS(endog = Joint['female_list'], exog = sm.add_constant(Joint[['treatment','numhh_list']]))
bal_ch_gender = gender.fit()
print(bal_ch_gender.summary())
print("p-value is 0.134, we cannot reject the null hypothesis--> Balance")

                            OLS Regression Results                            
Dep. Variable:            female_list   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     21.44
Date:                Thu, 20 May 2021   Prob (F-statistic):           4.98e-10
Time:                        20:24:01   Log-Likelihood:                -17758.
No. Observations:               24646   AIC:                         3.552e+04
Df Residuals:                   24643   BIC:                         3.555e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6074      0.010     61.420      0.0

In [57]:
s = sm.OLS(endog = Joint['self_list'], exog = sm.add_constant(Joint[['treatment','numhh_list']]))
bal_ch_self = s.fit()
print(bal_ch_self.summary())
print("p-value is 0.771, we cannot reject the null hypothesis--> Balance")

                            OLS Regression Results                            
Dep. Variable:              self_list   R-squared:                       0.443
Model:                            OLS   Adj. R-squared:                  0.443
Method:                 Least Squares   F-statistic:                     9811.
Date:                Thu, 20 May 2021   Prob (F-statistic):               0.00
Time:                        20:25:39   Log-Likelihood:                 1715.4
No. Observations:               24646   AIC:                            -3425.
Df Residuals:                   24643   BIC:                            -3400.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4915      0.004    332.379      0.0

In [58]:
english = sm.OLS(endog = Joint['english_list'], exog = sm.add_constant(Joint[['treatment','numhh_list']]))
bal_ch_english = english.fit()
print(bal_ch_english.summary())
print("p-value is 0.035, we reject the null hypothesis--> imbalanced")

                            OLS Regression Results                            
Dep. Variable:           english_list   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     949.1
Date:                Thu, 20 May 2021   Prob (F-statistic):               0.00
Time:                        20:28:22   Log-Likelihood:                -7617.3
No. Observations:               24646   AIC:                         1.524e+04
Df Residuals:                   24643   BIC:                         1.526e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1329      0.007    172.879      0.0

In [59]:
fd = sm.OLS(endog = Joint['first_day_list'], exog = sm.add_constant(Joint[['treatment','numhh_list']]))
bal_ch_fd = fd.fit()
print(bal_ch_fd.summary())
print("p-value is 0.103, we cannot reject the null hypothesis--> Balance")

                            OLS Regression Results                            
Dep. Variable:         first_day_list   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.370
Date:                Thu, 20 May 2021   Prob (F-statistic):              0.254
Time:                        20:31:04   Log-Likelihood:                -4478.2
No. Observations:               24646   AIC:                             8962.
Df Residuals:                   24643   BIC:                             8987.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0937      0.006     16.233      0.0

In [61]:
phone= sm.OLS(endog = Joint['have_phone_list'], exog = sm.add_constant(Joint[['treatment','numhh_list']]))
bal_ch_phone= phone.fit()
print(bal_ch_phone.summary())
print("p-value is 0.542, we cannot reject the null hypothesis--> Balance")

                            OLS Regression Results                            
Dep. Variable:        have_phone_list   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     31.28
Date:                Thu, 20 May 2021   Prob (F-statistic):           2.71e-14
Time:                        20:33:44   Log-Likelihood:                -8098.6
No. Observations:               24646   AIC:                         1.620e+04
Df Residuals:                   24643   BIC:                         1.623e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8197      0.007    122.675      0.0

In [62]:
pb= sm.OLS(endog = Joint['pobox_list'], exog = sm.add_constant(Joint[['treatment','numhh_list']]))
bal_ch_postbox= pb.fit()
print(bal_ch_postbox.summary())
print("p-value is 0.647, we cannot reject the null hypothesis--> Balance")

                            OLS Regression Results                            
Dep. Variable:             pobox_list   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     5.040
Date:                Thu, 20 May 2021   Prob (F-statistic):            0.00648
Time:                        20:33:59   Log-Likelihood:                 10162.
No. Observations:               24646   AIC:                        -2.032e+04
Df Residuals:                   24643   BIC:                        -2.029e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0357      0.003     11.211      0.0

In [ ]:
Balance Check Summary: While most of the covariate variables being balanced in the control and treatment group, the "english_list" shows there is a selection bias regarding the english level. Therefore, we need to adopt the 2SLS in order to have an unbiased estimation. 

# Model Fitting

In [66]:
# Fit the first-stage OLS and have the weak instrument test.
First_Stage = sm.OLS(endog = Joint['Enrollment'], exog = sm.add_constant(Joint[['treatment',"numhh_list"]]))
WIT = First_Stage.fit()
print(WIT.summary())

print(" [Based on the weak instrument test, the F-statistic of the first stage regression is 1006>10, so the IVs satisfy the strong first-stage assumption.]")

                            OLS Regression Results                            
Dep. Variable:             Enrollment   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     1006.
Date:                Thu, 20 May 2021   Prob (F-statistic):               0.00
Time:                        20:50:22   Log-Likelihood:                -13213.
No. Observations:               24646   AIC:                         2.643e+04
Df Residuals:                   24643   BIC:                         2.646e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1953      0.008     23.744      0.0

In [69]:
# Fit the second stage 
from linearmodels.iv import IV2SLS
IV_model = IV2SLS(dependent = Joint['any_visit'],endog = Joint['Enrollment'],\
                  exog = sm.add_constant(Joint["numhh_list"]),instruments=Joint['treatment'])
R_2sls = IV_model.fit()
print(R_2sls)

                          IV-2SLS Estimation Summary                          
Dep. Variable:              any_visit   R-squared:                      0.0300
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0299
No. Observations:               24646   F-statistic:                    413.87
Date:                Thu, May 20 2021   P-value (F-stat)                0.0000
Time:                        20:51:39   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.4890     0.0106     46.101     0.0000      0.4682      0.5098
numhh_list    -0.1364     0.0067    -20.244     0.00

In [72]:
# Fit the second stage on numbers of visit
IV_model_2 = IV2SLS(dependent = Joint['num_visit'],endog = Joint['Enrollment'],\
                  exog = sm.add_constant(Joint["numhh_list"]),instruments=Joint['treatment'])
R_2sls_2 = IV_model_2.fit()
print(R_2sls_2)

                          IV-2SLS Estimation Summary                          
Dep. Variable:              num_visit   R-squared:                      0.0220
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0219
No. Observations:               24622   F-statistic:                    468.38
Date:                Thu, May 20 2021   P-value (F-stat)                0.0000
Time:                        20:56:12   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          1.6197     0.0506     32.036     0.0000      1.5206      1.7188
numhh_list    -0.5866     0.0272    -21.596     0.00

* Summary: From the first 2SLS regression results, we see that there is a postive causal relationship between the increasing in visit and the Medicaid coverage. And the second 2SLS gives an even concrete support that one unit increase in Medicaid coverage results in 34.5% of incresase in overall emergency department use. Taubman et.al s' conclusion have been confirmed.